# 描述：

    报社等相关的机构，往往会遇到一个问题，就是别的机构使用他们的文章但没有标明来源，这种属于抄袭的现象。在本次任务中，我们将解决新华社的文章被抄袭引用的问题。
    在给定的数据集合中，存在一些新闻语料来自新华社，但是其来源并不是新华社，请设计技巧学习模型解决该问题（判别该文章是不是抄袭新华社的）。
    
    参考资料：
    https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import jieba
import re
import warnings
from sklearn.feature_extraction.text import TfidfVectorizer
%matplotlib
warnings.filterwarnings('ignore')

Using matplotlib backend: MacOSX


## Step1: 数据分析

In [2]:
# # 设置显示的最大列、宽等参数，消掉打印不完全中间的省略号
# pd.set_option('display.max_columns', 10000)
# pd.set_option('display.width', 10000)
# pd.set_option('display.max_colwidth', 10000)


In [3]:
# 导入数据
filename = '../data_sets/sqlResult_1558435.csv'
all_data = pd.read_csv(filename, encoding='gb18030')
all_data.head()

,id,author,source,content,feature,title,url
0,89617,NaN,快科技@http://www.kkj.cn/,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""37""...",小米MIUI 9首批机型曝光：共计15款,http://www.cnbeta.com/articles/tech/623597.htm
1,89616,NaN,快科技@http://www.kkj.cn/,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""15""...",骁龙835在Windows 10上的性能表现有望改善,http://www.cnbeta.com/articles/tech/623599.htm
2,89615,NaN,快科技@http://www.kkj.cn/,此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""18""...",一加手机5细节曝光：3300mAh、充半小时用1天,http://www.cnbeta.com/articles/tech/623601.htm
3,89614,NaN,新华社,这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n,"{""type"":""国际新闻"",""site"":""环球"",""commentNum"":""0"",""j...",葡森林火灾造成至少62人死亡 政府宣布进入紧急状态（组图）,http://world.huanqiu.com/hot/2017-06/10866126....
4,89613,胡淑丽_MN7479,深圳大件事,（原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走……）\r\n@深圳交警微博称：昨日清...,"{""type"":""新闻"",""site"":""网易热门"",""commentNum"":""978"",...",44岁女子约网友被拒暴雨中裸奔 交警为其披衣相随,http://news.163.com/17/0618/00/CN617P3Q0001875...


In [4]:
# 查看 table_labels
print(all_data.columns.values.tolist())

['id', 'author', 'source', 'content', 'feature', 'title', 'url']


In [5]:
# 查看整体数据的情况
print(all_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89611 entries, 0 to 89610
Data columns (total 7 columns):
id         89611 non-null int64
author     79396 non-null object
source     89609 non-null object
content    87054 non-null object
feature    89611 non-null object
title      89577 non-null object
url        87144 non-null object
dtypes: int64(1), object(6)
memory usage: 4.8+ MB
None


In [6]:
# 计算新华社文章所占的比例
xinhua_news = all_data[all_data['source'] == '新华社']
len(xinhua_news)/len(all_data)

0.8778051801676133

## Step2: 数据预处理

In [7]:
# 添加一列 label，来源于新华社的为1，其他为0
all_data['label'] = all_data['source'].apply(lambda x: 1 if x == '新华社' else 0)

In [8]:
print(all_data.columns.values.tolist())

['id', 'author', 'source', 'content', 'feature', 'title', 'url', 'label']


In [9]:
data = all_data[['content', 'label']]

In [10]:
data.head()

,content,label
0,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...,0
1,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...,0
2,此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n...,0
3,这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n,1
4,（原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走……）\r\n@深圳交警微博称：昨日清...,0


In [11]:
len(data)

89611

In [12]:
# 删除 content 为空的行
data_news = data.dropna(subset=['content'])
len(data_news)

87054

In [13]:
data_news.index = range(len(data_news))

In [14]:
data_news.head()

,content,label
0,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...,0
1,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...,0
2,此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n...,0
3,这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n,1
4,（原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走……）\r\n@深圳交警微博称：昨日清...,0


In [15]:
texts = data_news['content'] # 内容
label = data_news['label'] # 标签

## Step3: 文本向量化（使用tf_idf）

In [16]:
from tqdm import tqdm # 用于显示进度条的

In [17]:
for i in tqdm(range(10000000)):
    pass

100%|██████████| 10000000/10000000 [00:03<00:00, 3323918.89it/s]


In [18]:
# 切词
def get_contents(texts):
    contents = []
    for text in tqdm(texts):
        sentence = ''.join(re.findall(r'\w+', text))
        contents.append(' '.join(jieba.cut(sentence)))
    return contents

In [19]:
contents = get_contents(texts)

  0%|          | 0/87054 [00:00<?, ?it/s]Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/dp/9d1tlc2926x2djj4c05f3325ps5m7_/T/jieba.cache
Loading model cost 0.865 seconds.
Prefix dict has been built succesfully.
100%|██████████| 87054/87054 [02:59<00:00, 485.25it/s]


In [20]:
contents[0:2]

['此外 自 本周 6 月 12 日起 除 小米 手机 6 等 15 款 机型 外 其余 机型 已 暂停 更新 发布 含 开发 版 体验版 内测 稳定版 暂不受 影响 以 确保 工程师 可以 集中 全部 精力 进行 系统优化 工作 有人 猜测 这 也 是 将 精力 主要 用到 MIUI9 的 研发 之中 MIUI8 去年 5 月 发布 距今已有 一年 有余 也 是 时候 更新换代 了 当然 关于 MIUI9 的 确切 信息 我们 还是 等待 官方消息',
 '骁龙 835 作为 唯一 通过 Windows10 桌面 平台 认证 的 ARM 处理器 高通 强调 不会 因为 只 考虑 性能 而 去 屏蔽掉 小 核心 相反 他们 正 联手 微软 找到 一种 适合 桌面 平台 的 兼顾 性能 和 功耗 的 完美 方案 报道 称 微软 已经 拿到 了 一些 新 的 源码 以便 Windows10 更好 地 理解 biglittle 架构 资料 显示 骁龙 835 作为 一款 集成 了 CPUGPU 基带 蓝牙 WiFi 的 SoC 比 传统 的 Wintel 方案 可以 节省 至少 30 的 PCB 空间 按计划 今年 Q4 华硕 惠普 联想 将 首发 骁龙 835Win10 电脑 预计 均 是 二合一 形态 的 产品 当然 高通 骁龙 只是 个 开始 未来 也许 还 能 见到 三星 Exynos 联发科 华为 麒麟 小米 澎湃 等 进入 Windows10 桌面 平台']

In [21]:
vectorizer = TfidfVectorizer(max_features=300)
vectors = vectorizer.fit_transform(contents)

In [22]:
X = vectors.toarray()
y = label.tolist()

In [23]:
X.shape

(87054, 300)

## Step4: 构建不同的机器学习模型进行训练

## Step5: 分别在 traning_data, validation_data, test_data 上观察其相关 metric: recall, precision, f1等值， 并解释其含义

## Step6: 使用 test_data 检测模型的效果

## Step7: 调整模型参数，观测变化

## Step8: 不断调整参数，直到模型性能到达“某个”点时最优。

In [24]:
# 构建模型
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB,MultinomialNB

In [25]:
# 拆分数据集
X_train, X_test, Y_train, Y_test = train_test_split(X, y, random_state=50, test_size=0.15) # 拆成训练和测试集
x_train, x_valid, y_train, y_valid = train_test_split(X_train, Y_train, random_state=50, test_size=0.15) # 拆成训练和验证集

In [26]:
X_train.shape, X_test.shape

((73995, 300), (13059, 300))

### KNN

#### 默认 n_neighbors=5

In [27]:
knn = KNeighborsClassifier()
knn.fit(x_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform')

In [28]:
knn.score(x_valid, y_valid)

0.9204504504504505

In [29]:
y_pred = knn.predict(x_valid)
y_pred_prob = knn.predict_proba(x_valid)

In [30]:
precision_score(y_valid, y_pred)

0.9519944570919529

In [31]:
recall_score(y_valid, y_pred)

0.9602635782747604

In [32]:
f1_score(y_valid, y_pred)

0.9561111387245887

In [33]:
roc_auc_score(y_valid, y_pred_prob[:, 1])

0.9035646286974051

### n_neighbors=3

In [34]:
knn3 = KNeighborsClassifier(n_neighbors=3)
knn3.fit(x_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=3, p=2,
           weights='uniform')

In [35]:
knn3.score(x_valid, y_valid)

0.9218018018018018

In [36]:
y_pred = knn3.predict(x_valid)
y_pred_prob = knn3.predict_proba(x_valid)

In [37]:
precision_score(y_valid, y_pred)

0.9551243781094527

In [38]:
recall_score(y_valid, y_pred)

0.9583666134185304

In [39]:
f1_score(y_valid, y_pred)

0.9567427489285357

In [40]:
roc_auc_score(y_valid, y_pred_prob[:, 1])

0.880094247727621

### n_neighbors=7

In [41]:
knn7 = KNeighborsClassifier(n_neighbors=7)
knn7.fit(x_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=7, p=2,
           weights='uniform')

In [42]:
knn7.score(x_valid, y_valid)

0.913963963963964

In [43]:
y_pred = knn7.predict(x_valid)
y_pred_prob = knn7.predict_proba(x_valid)

In [44]:
precision_score(y_valid, y_pred)

0.9535489037941736

In [45]:
recall_score(y_valid, y_pred)

0.9509784345047924

In [46]:
f1_score(y_valid, y_pred)

0.9522619345163708

In [47]:
roc_auc_score(y_valid, y_pred_prob[:, 1])

0.9094703087605955

### n_neighbors=7, weights='distance'

In [48]:
knn_d = KNeighborsClassifier(n_neighbors=7, weights='distance')
knn_d.fit(x_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=7, p=2,
           weights='distance')

In [49]:
knn_d.score(x_valid, y_valid)

0.9238738738738739

In [50]:
y_pred = knn_d.predict(x_valid)
y_pred_prob = knn_d.predict_proba(x_valid)

In [51]:
precision_score(y_valid, y_pred)

0.9577717879604672

In [52]:
recall_score(y_valid, y_pred)

0.957867412140575

In [53]:
f1_score(y_valid, y_pred)

0.9578195976638547

In [54]:
roc_auc_score(y_valid, y_pred_prob[:, 1])

0.923288559338269

### n_neighbors=7, weights='distance', leaf_size=50

In [55]:
knn_s = KNeighborsClassifier(n_neighbors=7, weights='distance', leaf_size=50)
knn_s.fit(x_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=50, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=7, p=2,
           weights='distance')

In [56]:
knn_s.score(x_valid, y_valid)

0.9236036036036036

In [57]:
y_pred = knn_s.predict(x_valid)
y_pred_prob = knn_s.predict_proba(x_valid)

In [58]:
precision_score(y_valid, y_pred)

0.957667731629393

In [59]:
recall_score(y_valid, y_pred)

0.957667731629393

In [60]:
f1_score(y_valid, y_pred)

0.957667731629393

In [61]:
roc_auc_score(y_valid, y_pred_prob[:, 1])

0.9227932264097003

    小结：从上述验证结果可以看出，“n_neighbors=7, weights='distance', leaf_size=50” 时候效果较为好些。但是KNN运行速度太慢了~

### LogisticRegression

In [62]:
lr_classifier = LogisticRegression()
lr_classifier.fit(x_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [63]:
lr_classifier.score(x_valid, y_valid)

0.9772972972972973

In [64]:
y_pred = lr_classifier.predict(x_valid)
y_pred_prob = lr_classifier.predict_proba(x_valid)

In [65]:
precision_score(y_valid, y_pred)

0.9832706394773312

In [66]:
recall_score(y_valid, y_pred)

0.9917132587859425

In [67]:
f1_score(y_valid, y_pred)

0.9874739039665971

In [68]:
roc_auc_score(y_valid, y_pred_prob[:, 1])

0.9903634351089916

    random_state=50, solver=newton-cg

In [69]:
lr_classifier1 = LogisticRegression(random_state=50, solver='newton-cg')
lr_classifier1.fit(x_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=50, solver='newton-cg',
          tol=0.0001, verbose=0, warm_start=False)

In [70]:
lr_classifier1.score(x_valid, y_valid)

0.9772972972972973

In [71]:
y_pred = lr_classifier1.predict(x_valid)
y_pred_prob = lr_classifier1.predict_proba(x_valid)

In [72]:
precision_score(y_valid, y_pred)

0.9832706394773312

In [73]:
recall_score(y_valid, y_pred)

0.9917132587859425

In [74]:
f1_score(y_valid, y_pred)

0.9874739039665971

In [75]:
roc_auc_score(y_valid, y_pred_prob[:, 1])

0.9903621456591962

    小结：从结果上看出，貌似没有什么特别大的变化。

### Naive Bayes

    高斯朴素贝叶斯(sklearn.naive_bayes.GaussianNB(priors=None))

In [76]:
gnb_classifier = GaussianNB()
gnb_classifier.fit(x_train, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [77]:
gnb_classifier.score(x_valid, y_valid)

0.8027927927927928

In [78]:
y_pred = gnb_classifier.predict(x_valid)
y_pred_prob = gnb_classifier.predict_proba(x_valid)

In [79]:
precision_score(y_valid, y_pred)

0.9939416887542597

In [80]:
recall_score(y_valid, y_pred)

0.7862420127795527

In [81]:
f1_score(y_valid, y_pred)

0.8779753609454262

In [82]:
roc_auc_score(y_valid, y_pred_prob[:, 1])

0.912817213860038

    多项式分布贝叶斯(sklearn.naive_bayes.MultinomialNB(alpha=1.0, fit_prior=True, class_prior=None))

In [83]:
mnb_classifier = MultinomialNB()
mnb_classifier.fit(x_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [84]:
mnb_classifier.score(x_valid, y_valid)

0.8962162162162162

In [85]:
y_pred = mnb_classifier.predict(x_valid)
y_pred_prob = mnb_classifier.predict_proba(x_valid)

In [86]:
precision_score(y_valid, y_pred)

0.9552177485620378

In [87]:
recall_score(y_valid, y_pred)

0.9285143769968051

In [88]:
f1_score(y_valid, y_pred)

0.9416767922235723

In [89]:
roc_auc_score(y_valid, y_pred_prob[:, 1])

0.9209381686718225

    小结：从结果看出，MultinomialNB 要比 GaussianNB 效果好。

### 总结：

+ LR 效果比其他两种效果要好
+ KNN 速度是最慢的，效果比 Naive Bayes 稍好
+ Naive Bayes 速度比 KNN 快，但是效果相对最差

### 各模型在 test 数据集上的表现

#### KNN(n_neighbors=7, weights='distance', leaf_size=50)

In [90]:
model_info = "The model is {}, the parameters are {}."
result_info = '''
Test result: 
score = {}
ps = {}
rs = {}
f1 = {}
ras = {}
'''
x_test = X_test
y_test = Y_test

In [91]:
score = knn_s.score(x_test, y_test)
y_pred = knn_s.predict(x_test)
y_pred_prob = knn_s.predict_proba(x_test)
ps = precision_score(y_test, y_pred)
rs = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
ras = roc_auc_score(y_test, y_pred_prob[:, 1])
print(model_info.format("KNN", "(n_neighbors=7, weights='distance', leaf_size=50)"))
print(result_info.format(score, ps, rs, f1, ras))

The model is KNN, the parameters are (n_neighbors=7, weights='distance', leaf_size=50).

Test result: 
score = 0.9290910483191669
ps = 0.9585121602288984
rs = 0.9633764695931659
f1 = 0.9609381591158357
ras = 0.9240948680195779



#### LogisticRegression

In [92]:
score = lr_classifier.score(x_test, y_test)
y_pred = lr_classifier.predict(x_test)
y_pred_prob = lr_classifier.predict_proba(x_test)
ps = precision_score(y_test, y_pred)
rs = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
ras = roc_auc_score(y_test, y_pred_prob[:, 1])
print(model_info.format("LogisticRegression", "(default)"))
print(result_info.format(score, ps, rs, f1, ras))

The model is LogisticRegression, the parameters are (default).

Test result: 
score = 0.976338157592465
ps = 0.982645875251509
rs = 0.9913727480334941
f1 = 0.98699002147278
ras = 0.9918099204364704



#### Naive Bayes (MultinomialNB)

In [93]:
score = mnb_classifier.score(x_test, y_test)
y_pred = mnb_classifier.predict(x_test)
y_pred_prob = mnb_classifier.predict_proba(x_test)
ps = precision_score(y_test, y_pred)
rs = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
ras = roc_auc_score(y_test, y_pred_prob[:, 1])
print(model_info.format("Naive Bayes (MultinomialNB)", "(default)"))
print(result_info.format(score, ps, rs, f1, ras))

The model is Naive Bayes (MultinomialNB), the parameters are (default).

Test result: 
score = 0.8981545294432958
ps = 0.9542817559961901
rs = 0.9321661168908061
f1 = 0.9430943008728393
ras = 0.9200292707401815



#### 综上，下面选择 LogisticRegression 来检测文章是否抄袭新华社。

## Step9: 找出预测结果与实际结果相反的文章，作为抄袭的候选者。

In [94]:
# 预测所有的样本
score = lr_classifier.score(X, y)
y_pred = lr_classifier.predict(X)
y_pred_prob = lr_classifier.predict_proba(X)
ps = precision_score(y, y_pred)
rs = recall_score(y, y_pred)
f1 = f1_score(y, y_pred)
ras = roc_auc_score(y, y_pred_prob[:, 1])
print(model_info.format("LogisticRegression", "(default)"))
print(result_info.format(score, ps, rs, f1, ras))

The model is LogisticRegression, the parameters are (default).

Test result: 
score = 0.9777724171204081
ps = 0.9833194748910222
rs = 0.9922324913235275
f1 = 0.9877558768627203
ras = 0.9921231202146438



In [95]:
data_news.head()

,content,label
0,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...,0
1,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...,0
2,此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n...,0
3,这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n,1
4,（原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走……）\r\n@深圳交警微博称：昨日清...,0


In [96]:
len(data_news)

87054

In [97]:
len(y_pred)

87054

In [98]:
data_news['pred_label'] = y_pred

In [99]:
data_news.head()

,content,label,pred_label
0,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...,0,0
1,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...,0,0
2,此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n...,0,0
3,这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n,1,1
4,（原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走……）\r\n@深圳交警微博称：昨日清...,0,0


In [100]:
# 预测错误的个数
(data_news['label'] != data_news['pred_label']).sum()

1935

In [101]:
# 查看实际是0，预测为1。表示抄袭新华社的文章
copy_data = data_news[(data_news['label'] == 0) & (data_news['pred_label'] == 1)]

In [102]:
copy_data.head(10)

,content,label,pred_label
20,原标题：叙利亚被“袭机”事件惹怒俄罗斯 警告将瞄准美战机\r\n 海外网6月19日电 ...,0,1
37,切尔西球星法布雷加斯正与女友达涅拉-塞曼享受假期，在个人的INSTAGRAM上，小法发布...,0,1
51,戈壁的大漠黄沙曾掩埋了无数西域古道，而如今一条大漠天路正顽强地与黄沙“搏斗”，在乌兰布和、腾...,0,1
56,很快，不少人主动添加记者为好友，询问是否需要扫描软件，并声称这些扫描软件能够攻破摄像头的IP...,0,1
61,在QQ搜索栏输入“摄像头 破解”，就跳出了众多相关聊天群，记者随机加入了几个，发现聊天的内容...,0,1
63,2017年上海国际电影电视节，包括6月12日至16日举办的第二十三届上海电视节和6月17日至...,0,1
65,从百度、支付宝到App?“高考志愿填报”五花八门\r\n顾女士近来减少了刷朋友圈的频率，一有...,0,1
68,中新网6月23日电 6月22日，阿里巴巴旗下UC神马搜索主办的“U势 智赢”乘风大会在广州顺...,0,1
70,发现???共享单车“花式载人”??挡泥板当“后座”\r\n如今，共享单车种类繁多，有的共享单...,0,1
94,中新网6月23日电 日前，2017 金科“健康更好”品牌战略发布暨金科产业暨东软熙康战略合作...,0,1


## Step10: 总结上述过程，并分析以下问题：
### 1、什么是数据思维？
+ 任何事物的变化都可以用数据进行描述。反之，也能从已有的数据中也隐含了事物的变化过程，需要我们善于从数据的角度去发现一些规律，这样才能解决我们面临的问题。没有数据支撑的机器学习算法模型也纯粹是空谈。这就要求我们具有对数据一定的理解能力，善于从数据中去发现我们需要解决问题的规律。
### 2、什么是机器学习思维？
+ 机器学习是基于数据驱动的，它是一种对已有数据的进行不断归纳和总结，形成自己的一套解决某种问题的规则（y=f(x)）。而这种映射规则并非人为能够直接穷举的。
### 3、在涉嫌抄袭的文章中，使用 edit_distance 找出重复的文字与被修改过的文字。
+ 下面以 copy_data.head(0) 为例进行计算

In [103]:
copy_text = copy_data['content'].values[0]

In [104]:
print(copy_text)

　　原标题：叙利亚被“袭机”事件惹怒俄罗斯 警告将瞄准美战机
　　海外网6月19日电 当地时间6月19日，俄罗斯国防部对美国军方击落叙利亚飞机一事作出反击，宣布停止执行俄美两国签署的“在叙飞行安全备忘录”，并称以后美国领导的国际联军所有的战机，都是俄罗斯军方监控与瞄准的目标，叙利亚局势进一步复杂化。
　　据纽约时报消息，由于美国军方今日击落了一架叙利亚军机，俄罗斯国防部发布消息，自6月19日起暂停执行俄美间在叙利亚领空“防止空中事件和保障行动期间飞行安全”的相互谅解备忘录。要求美方指挥部对此事件进行彻查，结果与俄方共享。
　　公告称：“俄空军在叙利亚领空执行任务的地区里，幼发拉底河西岸发现的任何飞行物，包括美国领导的国际联军的飞机和无人机，都将是俄罗斯军方地面和空中防空武器监控与瞄准的目标。”
　　据叙利亚军方声明，当地时间6月19日，一架政府军机正前往拉卡（Raqqa）市，准备对盘踞于此的IS武装分子进行打击，却突然遭到美军袭击，飞行员至今失踪。声明称：“这次袭击发生的时机，是在叙利亚政府及其盟国的军队在与IS恐怖分子的战斗中获得优势的情况下发生的，本来这些恐怖分子已经在叙利亚的沙漠中节节败退。”
　　此次“袭机”事件“惹怒”了俄罗斯，俄罗斯参议院国防委员会副主席弗朗茨·克莱琴谢夫（Frants Klintsevich）称美军的行动是“挑衅行为”，实际上是对叙利亚的“军事侵略”。
　　该部门认为，美军“故意不履行双方2015年签署的“安全备忘录”中规定的义务，因此宣布暂停与美军在该框架下的合作。据报道，该协议一直是美俄两国军队协调在该地区的军事活动的关键。俄罗斯、美国、叙利亚、土耳其等国家在叙利亚的诉求经常是相冲突的，该协议就在其中起到调和作用。在特朗普四月下令袭击叙利亚空军之后，俄方表示将暂停协议，不过几个星期之后又重启，这次时隔两月后再被中断。
　　俄罗斯外长拉夫罗夫在回应记者时也表示，“涉及到叙利亚地面所发生的事情，毫无疑问，我们认为有必要尊重叙利亚的主权和领土完整，这是联合国2254号决议和其他文件规定的。因此，任何地面行动，包括实施军事行动的参与方，需得到大马士革的许可。”（编译/海外网 杨佳）



In [105]:
xinhua_news = data_news[data_news['label'] == 1]['content'].tolist()

In [106]:
xinhua_news[0]

'这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n'

In [107]:
# s -> t
def edit_distance(s, t):
    len1, len2 = len(s), len(t)
    if len1 == 0 and len2 == 0: return 0
    if len2 == 0: return len1
    if len1 == 0: return len2
    solution = {}
    dp = [[0] * (len2+1) for _ in range(len1+1)]
    for i in range(1, len2+1):
        dp[0][i] = i
    for i in range(1, len1+1):
        dp[i][0] = i
    for i in range(1, len1+1):
        for j in range(1, len2+1):
            if s[i-1] == t[j-1]:
                dp[i][j] = dp[i-1][j-1]
            else:
                options = [(dp[i-1][j-1], 'sub'), (dp[i-1][j], 'del'), (dp[i][j-1], 'add')]
                tmp = min(options)
                dp[i][j] = tmp[0] + 1
    return dp[-1][-1]

In [ ]:
def setEditDistance(text):
    return edit_distance(copy_text, text)

In [ ]:
data_news['edit_dis'] = data_news['content'].apply(lambda x: setEditDistance(x))

#### 上面这步太卡了，没有算不出来 :(

In [ ]:
# 在上一步的基础上，找出与 copy_text 编辑距离最短的文章
sim_text = data_news[data_news['edit_dis'] == min(data_news['edit_dis'])]['content'].values[0]

#### 下面我想从 X 中直接用余弦相似度的方法求得与当前 copy_text 最相似的文章

    1、从 X 中找出 copy_text 对应的向量 X_copy
    2、求解 X_copy 与 X 中对应新华社的文章向量相似度最高的向量 X_sim，即为新华社被抄袭的文章的向量
    3、再根据 X_sim 找到对应的文章 sim_text，即为被抄袭的文章

In [109]:
len(X)

87054

In [110]:
# 需要重新设置一下所有==索引，因为把 content 为空的去掉了，这样就能对应上 X 的索引
data_news.index = range(len(data_news))

In [111]:
copy_index = copy_data.index[0]
X_copy = X[copy_index]

In [112]:
# 求解两个向量的余弦相似度
def cos_sim(vector_a, vector_b):
    """
    计算两个向量之间的余弦相似度
    :param vector_a: 向量 a 
    :param vector_b: 向量 b
    :return: sim
    """
    vector_a = np.mat(vector_a)
    vector_b = np.mat(vector_b)
    num = float(vector_a * vector_b.T)
    denom = np.linalg.norm(vector_a) * np.linalg.norm(vector_b)
    cos = num / denom
    sim = 0.5 + 0.5 * cos
    return sim

In [113]:
def getMostSimText(X_copy):
    sim = 0.
    sim_idx = 0
    for idx in tqdm(range(len(X))):
        _sim = cos_sim(X[idx], X_copy)
        if y[idx] == 1 and _sim > sim:
            sim = _sim
            sim_idx = idx
    return data_news.loc[sim_idx]['content'], sim, sim_idx

In [114]:
sim_text, sim, sim_idx = getMostSimText(X_copy)

100%|██████████| 87054/87054 [00:06<00:00, 12953.75it/s]


In [115]:
(sim, sim_idx)

(0.8385285357085541, 11225)

In [116]:
print(sim_text)

　　新华社北京４月４日电（国际观察）圣彼得堡地铁爆炸案凸显俄反恐新威胁\n　　新华社记者孙萍　吴刚\n　　俄罗斯“北方之都”圣彼得堡３日发生地铁爆炸事件，已造成１４人死亡。爆炸发生时，俄总统普京正在圣彼得堡参加外交活动。俄总检察院已将这一爆炸事件定性为恐怖袭击。\n　　俄警方初步调查结果显示，爆炸嫌疑人很可能与宗教极端组织有关联。分析人士认为，自从俄罗斯在中东地区展开大规模反恐行动后，国际恐怖主义组织正成为俄面临的新安全威胁。\n　　（小标题）打破平静\n　　在一向风平浪静的圣彼得堡，这一突如其来的地铁爆炸案令当地居民感到震惊和茫然。自２０１０年以来，俄罗斯发生的多起公交系统遭袭事件主要集中在首都莫斯科和靠近高加索地区的伏尔加格勒，其中２０１０年的莫斯科地铁爆炸案导致４０人死亡。自２０１３年伏尔加格勒火车站爆炸案以来，俄境内已有三年多未发生针对公交系统的严重恐袭事件。\n　　俄罗斯联邦侦查委员会已经将袭击定性为恐怖事件，但克里姆林宫表态相对谨慎。俄总统新闻秘书佩斯科夫说，尽管有明显的恐袭迹象，但还需要调查所有可能性。\n　　从爆炸威力等细节判断，俄罗斯专家倾向于认为这是一起有预谋的恐袭事件。考虑到当时普京恰好身处圣彼得堡，这起事件的政治色彩也成为舆论关注的焦点。\n　　爆炸发生后，普京立即召开俄罗斯安全部门会议，听取了有关调查初步结果的汇报。随后，他来到遇袭地点附近一个地铁站的入口处摆放鲜花，向遇难者表示哀悼。\n　　事件发生后，一些国家和国际组织对俄政府和人民表示慰问。美国总统特朗普３日与普京通电话，表达了对俄罗斯人民的支持。对于特朗普传递的“团结”意愿，普京表示感谢。双方还强调了联合打击恐怖主义的必要性。\n　　（小标题）与“伊斯兰国”有关？\n　　目前，俄安全部门尚未完全确定袭击者身份和动机。俄罗斯媒体４日援引圣彼得堡安全部门的消息说，犯罪嫌疑人身份初步判断为中亚人，可能与中东极端分子有关联。\n　　俄罗斯舆论普遍认为，这起事件可能与“伊斯兰国”等极端组织有关联。\n　　俄罗斯国际问题专家苏斯洛夫认为，这起袭击可能与俄在叙利亚的反恐行动有关。国际反恐训练协会会长林德尔说，这起事件与俄罗斯在国际舞台上的强硬表现有关，包括在国际反恐领域的行动。\n　　车臣战争结束后，俄罗斯一直深受北高加索地区的分离主义组织之害，多次遭遇恐怖袭击。从２００４年的别斯兰人质事件到２０

In [117]:
print(copy_text)

　　原标题：叙利亚被“袭机”事件惹怒俄罗斯 警告将瞄准美战机
　　海外网6月19日电 当地时间6月19日，俄罗斯国防部对美国军方击落叙利亚飞机一事作出反击，宣布停止执行俄美两国签署的“在叙飞行安全备忘录”，并称以后美国领导的国际联军所有的战机，都是俄罗斯军方监控与瞄准的目标，叙利亚局势进一步复杂化。
　　据纽约时报消息，由于美国军方今日击落了一架叙利亚军机，俄罗斯国防部发布消息，自6月19日起暂停执行俄美间在叙利亚领空“防止空中事件和保障行动期间飞行安全”的相互谅解备忘录。要求美方指挥部对此事件进行彻查，结果与俄方共享。
　　公告称：“俄空军在叙利亚领空执行任务的地区里，幼发拉底河西岸发现的任何飞行物，包括美国领导的国际联军的飞机和无人机，都将是俄罗斯军方地面和空中防空武器监控与瞄准的目标。”
　　据叙利亚军方声明，当地时间6月19日，一架政府军机正前往拉卡（Raqqa）市，准备对盘踞于此的IS武装分子进行打击，却突然遭到美军袭击，飞行员至今失踪。声明称：“这次袭击发生的时机，是在叙利亚政府及其盟国的军队在与IS恐怖分子的战斗中获得优势的情况下发生的，本来这些恐怖分子已经在叙利亚的沙漠中节节败退。”
　　此次“袭机”事件“惹怒”了俄罗斯，俄罗斯参议院国防委员会副主席弗朗茨·克莱琴谢夫（Frants Klintsevich）称美军的行动是“挑衅行为”，实际上是对叙利亚的“军事侵略”。
　　该部门认为，美军“故意不履行双方2015年签署的“安全备忘录”中规定的义务，因此宣布暂停与美军在该框架下的合作。据报道，该协议一直是美俄两国军队协调在该地区的军事活动的关键。俄罗斯、美国、叙利亚、土耳其等国家在叙利亚的诉求经常是相冲突的，该协议就在其中起到调和作用。在特朗普四月下令袭击叙利亚空军之后，俄方表示将暂停协议，不过几个星期之后又重启，这次时隔两月后再被中断。
　　俄罗斯外长拉夫罗夫在回应记者时也表示，“涉及到叙利亚地面所发生的事情，毫无疑问，我们认为有必要尊重叙利亚的主权和领土完整，这是联合国2254号决议和其他文件规定的。因此，任何地面行动，包括实施军事行动的参与方，需得到大马士革的许可。”（编译/海外网 杨佳）



### 总结：
### 上面结果看起来差不多吧。相似度 0.83，肉眼看貌似有点像。特别提示：最后根据索引提取文章的时候，一定要先重置一下原来的索引，因为在数据预处理的时候删除了一些 content 为空的文章，导致索引不再是连续的。

## 上面存在一个问题，因为数据是imbalance的。同时训练数据不一定要用全量的数据，将在下次作业中改进